In [1]:
# Install packages
import os
from dotenv import load_dotenv
import pandas as pd
import tkinter as tk
from sqlalchemy import text, create_engine, MetaData, Table, update, and_, Update, insert
from sqlalchemy.orm import Session

# Import utils
from etl.utils.utils import extract_query, load_query

In [2]:
# load .env
load_dotenv()

# Create db connection string
db_username = os.getenv("DB_USERNAME")
db_password = os.getenv("DB_PASSWORD")
db_host = os.getenv("DB_HOST")
db_port = os.getenv("DB_PORT")
db_conn_str = f"mysql+pymysql://{db_username}:{db_password}@{db_host}:{db_port}/tsx_composite_index"

# Set name for table in database
staging_table = "ticker_staging_table"
company_table = "company"
ticker_table = "ticker_history"

# Create database engine
db_engine = create_engine(db_conn_str)

# Load metadata container
metadata = MetaData()

In [3]:
# Pull data from staging table
staging_df = extract_query(table_name=staging_table, engine=db_engine)

Connection Successful!
ticker_staging_table loaded successfully!


In [4]:
# Pull data from current ticker table
cur_tickers_query = text("""
SELECT company_id, tsx_ticker, company_name
FROM ticker_history
WHERE end_date IS NOT NULL
""")

tickers_df = extract_query(sql_query=cur_tickers_query, engine=db_engine)

Connection Successful!
SQL script executed successfully!


In [5]:
# Get date_created from staging table
date_created = staging_df['date_created'][0]

In [6]:
# Outer join the two dataframes
full_df = tickers_df.merge(staging_df, how = "outer", left_on = ["tsx_ticker", "company_name"], right_on = ["company_s", "company_n"])

In [14]:
# If there are no nulls, we have matched an existing row, and edits are needed
# If there are nulls, segment into the nulls from tsx_ticker and company_name vs company_s and company_n. We then have several cases:
### For the nulls in tsx_ticker and comp_name, check if the company has had a name/ticker change:
###### If yes: match to the corresponding row with null company_s/company_n. Then, set end_date = date_created, end_reason = 'C' for those company_ids on db table.
###### Finally, insert new rows with company_id, company_s, company_n, yfinance_ticker (computed) and record_created
###### If no: for leftover rows with null in company_s/company_n, set end_date = date_created, end_reason = 'R' for those company_ids on db table.
###### For rows left with null in tsx_ticker/company_name, first insert a new row into the company table with company_name.
###### Then pull rows from that table with name in company_n and date_created = date_created, and insert them into the ticker table with
###### company_id, company_s, company_n, yfinance_ticker (computed) and record_created
df_left = full_df[full_df["company_n"].isna() & full_df["company_s"].isna()][["company_id", "tsx_ticker", "company_name"]]
df_right = full_df[full_df["company_id"].isna() & full_df["tsx_ticker"].isna()][["company_s", "company_n", "date_created"]]
df_matches = pd.DataFrame()

In [16]:
df_right

,company_s,company_n,date_created
0,AAV,Advantage Energy Ltd.,2025-11-30
1,ABX,Barrick Mining Corporation,2025-11-30
2,AC,Air Canada Voting and Variable Voting Shares,2025-11-30
3,ACO.X,ATCO Ltd. Class I Non-voting Shares,2025-11-30
4,AEM,Agnico Eagle Mines Limited,2025-11-30
...,...,...,...
206,WN,George Weston Limited,2025-11-30
207,WPK,Winpak Ltd.,2025-11-30
208,WPM,Wheaton Precious Metals Corp.,2025-11-30
209,WSP,WSP Global Inc.,2025-11-30


In [17]:
# Start matches UI if there are rows in both dfs, and let user match rows if necessary
if len(df_left) > 0 and len(df_right) > 0:
    def match_rows():
        global df_left, df_right, df_matches

        left_idx = left_listbox.curselection()
        right_idx = right_listbox.curselection()
        if not left_idx or not right_idx:
            return  # require a selection on both sides

        # Extract full rows
        left_row = df_left.iloc[left_idx[0]]
        right_row = df_right.iloc[right_idx[0]]

        # Combine into one row
        combined = pd.concat([left_row, right_row])

        # Append to matches DataFrame
        df_matches = pd.concat([df_matches, combined.to_frame().T], ignore_index=True)

        # Show match in Matches window
        match_listbox.insert(
            tk.END,
            f"{left_row['tsx_ticker']} - {left_row['company_name']} ↔ {right_row['company_s']} - {right_row['company_n']}"
        )

        # Drop matched rows from original DataFrames
        df_left = df_left.drop(left_row.name).reset_index(drop=True)
        df_right = df_right.drop(right_row.name).reset_index(drop=True)

        # Remove matched rows from listboxes
        left_listbox.delete(left_idx[0])
        right_listbox.delete(right_idx[0])

        # Clear selections
        left_listbox.selection_clear(0, tk.END)
        right_listbox.selection_clear(0, tk.END)

    def finish_process():
        # Print matches
        print("The following rows were matched:")
        print(df_matches)

        root.destroy()  # close the GUI cleanly

    root = tk.Tk()
    root.title("Row Matcher: Match rows if necessary")

    # Left Listbox
    left_listbox = tk.Listbox(root, selectmode="browse", exportselection=False)
    for _, row in df_left.iterrows():
        left_listbox.insert(tk.END, f"{row['tsx_ticker']} - {row['company_name']}")
    left_listbox.pack(side="left", padx=10, pady=10, fill="y")

    # Right Listbox
    right_listbox = tk.Listbox(root, selectmode="browse", exportselection=False)
    for _, row in df_right.iterrows():
        right_listbox.insert(tk.END, f"{row['company_s']} - {row['company_n']}")
    right_listbox.pack(side="right", padx=10, pady=10, fill="y")

    # Match button
    btn = tk.Button(root, text="Match", command=match_rows)
    btn.pack(pady=10)

    # Finished button
    finish_btn = tk.Button(root, text="Finished", command=finish_process, bg="lightgreen")
    finish_btn.pack(pady=10)

    # Matches window
    matches_win = tk.Toplevel(root)
    matches_win.title("Matches")
    match_listbox = tk.Listbox(matches_win, width=60, exportselection=False)
    match_listbox.pack(padx=10, pady=10, fill="both")

    root.mainloop()

In [18]:
# Define function to construct a statement to invalidate rows based on matching columns
def invalidate_rows(table: Table, row: pd.Series, date_created: str, end_reason: str) -> Update:
    # Define statement
    stmt = (
            update(table)
            .where(
                and_(
                    table.c.company_id == row["company_id"],
                    table.c.tsx_ticker == row["tsx_ticker"],
                    table.c.company_name == row["company_name"],
                    table.c.end_date.is_(None)
                )
            )
            .values(
                end_date = date_created,
                end_reason = end_reason
            )
        )

    return stmt

In [19]:
# Set end_date for remaining rows in left table and matched rows
ticker_table_db = Table(ticker_table, metadata, autoload_with=db_engine)

# Begin engine session
with db_engine.begin() as conn:
    # First, invalidate the rows for the current name/tickers in the matched table
    for _, row in df_matches.iterrows():
        # Create statement to invalidate rows
        stmt = invalidate_rows(table=ticker_table_db, row=row, date_created=date_created, end_reason="C")

        # Execute transaction
        conn.execute(stmt)

    # Next, Invalidate each row in df_left
    for _, row in df_left.iterrows():
        # Create statement to invalidate rows
        stmt = invalidate_rows(table=ticker_table_db, row=row, date_created=date_created, end_reason="R")

        # Execute transaction
        conn.execute(stmt)

In [21]:
# Insert rows for matches (new tickers/names) into table, if any exist
if len(df_matches) > 0:
    # Drop columns
    df_matches.drop(columns = ["tsx_ticker", "company_name"], inplace=True)

    # Rename columns
    df_matches.rename(columns = {"company_s": "tsx_ticker", "company_n": "company_name", "date_created": "start_date"}, inplace=True)

    # Generate yfinance ticker
    df_matches["yfinance_ticker"] = df_matches["tsx_ticker"].str.replace(".", "-") + ".TO"

    # Reorder columns
    df_matches = df_matches[["company_id", "tsx_ticker", "company_name", "yfinance_ticker", "start_date"]]

    # Insert into database
    load_query(table_name = ticker_table, df = df_matches, engine = db_engine)

In [22]:
# Insert rows for new companies into company table

# First, define table using Table class
company_table_db = Table(company_table, metadata, autoload_with=db_engine)

# Create empty list for ids
ids = []

# Create session, and store ids
with Session(db_engine) as session:
    for _, row in df_right.iterrows():
        stmt = insert(company_table_db).values({"initial_name":row["company_n"], "date_created": row["date_created"]})
        result = session.execute(stmt)
        session.commit()
        ids.append(result.inserted_primary_key[0])

# Add the ids to the dataframe
df_right["company_id"] = ids

In [24]:
# Finally, insert rows for new companies into ticker table

# Rename columns
df_right.rename(columns = {"company_s": "tsx_ticker", "company_n": "company_name", "date_created": "start_date"}, inplace=True)

# Add column for yfinance_tickers
df_right["yfinance_ticker"] = df_right["tsx_ticker"].str.replace(".", "-") + ".TO"

# Reorder columns
df_right = df_right[["company_id", "tsx_ticker", "company_name", "yfinance_ticker", "start_date"]]

# Insert into db
load_query(table_name = ticker_table, df = df_right, engine = db_engine)

211 rows uploaded successfully to ticker_history.
